<a href="https://colab.research.google.com/github/Mike-Wazovsky/JetBrain-emotion-classification/blob/main/other_implementations/JetBrains_other_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Add project path to the PYTHONPATH

import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())

## Load Dataset

Load the emotion labeled dataset

In [4]:
from pathlib import Path
import pandas as pd

!pip install nlp

from nlp import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.9 MB/s eta 0:00:00


In [8]:
dataset_path = Path('fb_sentiment.csv').resolve()
dataset = pd.read_csv(dataset_path)
dataset['label'] = dataset['Label']
dataset['text'] = dataset['FBPost']

In [10]:
dataset.head()

,Unnamed: 0,FBPost,Label,label,text
0,0,Drug Runners and a U.S. Senator have somethin...,O,O,Drug Runners and a U.S. Senator have somethin...
1,1,"Heres a single, to add, to Kindle. Just read t...",O,O,"Heres a single, to add, to Kindle. Just read t..."
2,2,If you tire of Non-Fiction.. Check out http://...,O,O,If you tire of Non-Fiction.. Check out http://...
3,3,Ghost of Round Island is supposedly nonfiction.,O,O,Ghost of Round Island is supposedly nonfiction.
4,4,Why is Barnes and Nobles version of the Kindle...,N,N,Why is Barnes and Nobles version of the Kindle...


## Tokenize

Transform the text corpus to a vector representation

- **num_words**: Number of words to use

In [11]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

In [13]:
num_words = 10000

tokenizer = Tokenizer(num_words=num_words, lower=True)
tokenizer.fit_on_texts(dataset.text)

file_to_save = Path('tokenizer.pickle').resolve()
with file_to_save.open('wb') as file:
    pickle.dump(tokenizer, file)

## Split data

Split the dataset in train and validation data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
data = dataset.copy()

train = pd.DataFrame(columns=['label', 'text'])
validation = pd.DataFrame(columns=['label', 'text'])
for label in data.label.unique():
    label_data = data[data.label == label]
    train_data, validation_data = train_test_split(label_data, test_size=0.3)
    train = pd.concat([train, train_data])
    validation = pd.concat([validation, validation_data])

## Model

Define the **LSTM** + **CNN** model

In [16]:
from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Conv1D, Dense, concatenate
from tensorflow.keras.models import Model

In [17]:
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
num_classes = len(data.label.unique())
embedding_dim = 500
input_length = 100
lstm_units = 128
lstm_dropout = 0.1
recurrent_dropout = 0.1
spatial_dropout=0.2
filters=64
kernel_size=3

In [18]:
input_layer = Input(shape=(input_length,))
output_layer = Embedding(
  input_dim=input_dim,
  output_dim=embedding_dim,
  input_shape=(input_length,)
)(input_layer)

output_layer = SpatialDropout1D(spatial_dropout)(output_layer)

output_layer = Bidirectional(
LSTM(lstm_units, return_sequences=True,
     dropout=lstm_dropout, recurrent_dropout=recurrent_dropout)
)(output_layer)
output_layer = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                    kernel_initializer='glorot_uniform')(output_layer)

avg_pool = GlobalAveragePooling1D()(output_layer)
max_pool = GlobalMaxPooling1D()(output_layer)
output_layer = concatenate([avg_pool, max_pool])

output_layer = Dense(num_classes, activation='softmax')(output_layer)

model = Model(input_layer, output_layer)

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 500)     1464000     ['input_1[0][0]']                
                                                                                                  
 spatial_dropout1d (SpatialDrop  (None, 100, 500)    0           ['embedding[0][0]']              
 out1D)                                                                                           
                                                                                                  
 bidirectional (Bidirectional)  (None, 100, 256)     644096      ['spatial_dropout1d[0][0]']  

## Prepare the data

Prepare the model input data

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

In [23]:
train_sequences = [text.split() for text in train.text]
validation_sequences = [text.split() for text in validation.text]
list_tokenized_train = tokenizer.texts_to_sequences(train_sequences)
list_tokenized_validation = tokenizer.texts_to_sequences(validation_sequences)
x_train = pad_sequences(list_tokenized_train, maxlen=input_length)
x_validation = pad_sequences(list_tokenized_validation, maxlen=input_length)

encoder = LabelBinarizer()
encoder.fit(data.label.unique())

encoder_path = Path('encoder.pickle')
with encoder_path.open('wb') as file:
    pickle.dump(encoder, file)

y_train = encoder.transform(train.label)
y_validation = encoder.transform(validation.label)

## Train model

Do the training process with the given data

In [24]:
batch_size = 128
epochs = 1

In [25]:
model.fit(
    x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_validation, y_validation)
)

6/6 [==============================] - 30s 3s/step - loss: 0.9353 - accuracy: 0.6295 - val_loss: 0.8692 - val_accuracy: 0.6412


In [ ]:
model_file = Path('../models/emotion_recognition/model_weights.h5').resolve()
model.save_weights(model_file.as_posix())